In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import mahotas as mh
import json

In [3]:
import cactas as C

/home/jiehyun.kim001/miniconda3/envs/O/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
DATAPATH = '/raid/mpsych/CACTAS/DATA/ESUS/'

In [5]:
images = []
labels = []

g_symp_i_patches = []
g_symp_l_patches = []
g_asymp_i_patches = []
g_asymp_l_patches = []

image_patches = []
label_patches = []

In [6]:
with open("image_data_v3.json", "r") as fp:
    image_data = json.load(fp)
with open("label_data_v3.json", "r") as fp:
    label_data = json.load(fp)

In [7]:
with open(DATAPATH+'../ESUS_stroke_side.csv', 'r') as f:
    stroke_sides = f.read().replace('ESUS ','').split('\n')

STROKE_SIDES = {}
    
for l in stroke_sides[1:]:
    l_s = l.split(',')
    
    if len(l_s) != 5:
        continue
    
    id_ = l_s[0]
    age = l_s[1]
    sex = l_s[2]
    race = l_s[3]
    side = l_s[4]
    STROKE_SIDES[id_] = side

In [8]:
images, labels = C.Util.load(DATAPATH)

In [9]:
for key, val in image_data.items():
    stroke_side = STROKE_SIDES[key]
    print('file name', key)
    print('Stroke side', stroke_side)

file name 2
Stroke side R
file name 5
Stroke side R
file name 6
Stroke side L
file name 7
Stroke side L
file name 9
Stroke side R
file name 10
Stroke side L
file name 12
Stroke side L
file name 13
Stroke side L
file name 14
Stroke side L
file name 15
Stroke side L
file name 16
Stroke side L
file name 17
Stroke side R
file name 18
Stroke side L
file name 21
Stroke side R
file name 22
Stroke side L
file name 23
Stroke side L
file name 24
Stroke side R
file name 25
Stroke side L
file name 26
Stroke side R
file name 27
Stroke side R
file name 29
Stroke side R
file name 31
Stroke side L
file name 32
Stroke side R
file name 33
Stroke side R
file name 35
Stroke side L
file name 36
Stroke side R
file name 39
Stroke side L
file name 40
Stroke side R
file name 41
Stroke side R
file name 45
Stroke side L
file name 46
Stroke side R
file name 47
Stroke side L
file name 48
Stroke side L
file name 49
Stroke side R
file name 50
Stroke side R
file name 51
Stroke side L
file name 52
Stroke side R
file n

In [10]:
# normalize grayscale to 0..1
images, labels = C.Util.normalize(images, labels)

# for each scan
symp_i_patches = []
symp_l_patches = []
asymp_i_patches = []
asymp_l_patches = []

In [11]:
for i in range(labels.shape[0]):
    bbox = mh.bbox(labels[i])
    
    #print(bbox)

    location = ''
    if (512-bbox[0] < 256):
        location = 'L' # patient view
    else:
        location = 'R' # patient view
        
    increase_x = 10
    increase_y = 10
    increase_z=0
    target_size=None
    just_z=False
    
    if target_size:
        widthY = bbox[1]-bbox[0]
        widthX = bbox[3]-bbox[2]

        increase_x = (target_size - widthX) // 2
        increase_y = (target_size - widthY) // 2

        bufferY = 0
    if (bbox[0]-increase_y + bbox[1]+increase_y) != target_size:
        bufferY = 1

        bufferX = 0
    if (bbox[2]-increase_x + bbox[3]+increase_x) != target_size:
        bufferX = 1


    if just_z:
        increase_xy = 0
        bbox[0] = 0
        bbox[1] = image.shape[0]
        bbox[2] = 0
        bbox[3] = image.shape[1]


    # crop label and image according to bbox but make it a little larger
    label_cropped = labels[bbox[0]-increase_y:bbox[1]+increase_y+bufferY, 
                          bbox[2]-increase_x:bbox[3]+increase_x+bufferX,
                          bbox[4]-increase_z:bbox[5]+increase_z]
    image_cropped = images[bbox[0]-increase_y:bbox[1]+increase_y+bufferY, 
                          bbox[2]-increase_x:bbox[3]+increase_x+bufferX,
                          bbox[4]-increase_z:bbox[5]+increase_z]
    
    if location == stroke_side:
        symp_i_patches.append(image_cropped)
        symp_l_patches.append(label_cropped)
    else:
        asymp_i_patches.append(image_cropped)
        asymp_l_patches.append(label_cropped)


In [13]:
l = symp_i_patches

count = len(l)
#print(count)
#print(l[0].shape) #(25, 27, 1, 1)
d_y = l[0].shape[0]
d_x = l[0].shape[1]
#print(d_y, d_x)

out = np.zeros((d_y, d_x, count), dtype=l[0].dtype)
#print(out.shape)
for i in range(count):
    c_d_y = min(d_y, l[i].shape[0])
    c_d_x = min(d_x, l[i].shape[1])
    #print(c_d_y)
    #print(c_d_x)

    out[0:c_d_y,0:c_d_x,i] = l[i][0:c_d_y,0:c_d_x,0]


ValueError: could not broadcast input array from shape (25,27,1) into shape (25,27)

In [12]:
def list_to_array(l):
    '''assuming all elements same size
    '''
    count = len(l)

    d_y = l[0].shape[0]
    d_x = l[0].shape[1]

    out = np.zeros((d_y, d_x, count), dtype=l[0].dtype)

    for i in range(count):
        c_d_y = min(d_y, l[i].shape[0])
        c_d_x = min(d_x, l[i].shape[1])

        out[0:c_d_y,0:c_d_x,i] = l[i][0:c_d_y,0:c_d_x,0]

    return out

In [13]:
def view(image, label, alpha=0.7, title=None, vmin=-600, vmax=1000):
    masked = np.ma.masked_where(label==0, label)

    if image.ndim == 3:
        slices = image.shape[2]
    else:
        slices = 1

    fig = plt.figure(figsize=(slices,3))

    if title:
        fig.suptitle(title, fontsize=12, fontweight='bold')

    for i in range(slices):
        plt.subplot(1, slices, i+1)

        plt.axis('off')

        if slices > 1:
            plt.imshow(image[:,:,i], cmap='gray', vmin=vmin, vmax=vmax)
            plt.imshow(masked[:,:,i], cmap='jet', interpolation='none', alpha=alpha) 
        else:
            plt.imshow(image, cmap='gray', vmin=vmin, vmax=vmax)
            plt.imshow(masked, cmap='jet', interpolation='none', alpha=alpha) 

In [14]:
for key, val in image_data.items():
    ### plot
    if len(symp_i_patches) > 0:
        symp_i_patches_a = list_to_array(symp_i_patches)
        view(symp_i_patches_a, symp_i_patches_a, vmin=0, vmax=1, alpha=0, title=d+' symp.' )
    else:
        print(key, 'no symp plaque')

    if len(asymp_i_patches) > 0:
        asymp_i_patches_a = list_to_array(asymp_i_patches)
        view(asymp_i_patches_a, asymp_i_patches_a, vmin=0, vmax=1, alpha=0, title=d+' asymp.' )
    else:
        print(key, 'no asymp plaque')

ValueError: could not broadcast input array from shape (25,27,1) into shape (25,27)